# This notebook is for eda on the May data

#### Import needed modules

In [2]:
import pandas as pd

#### Import .csv to DataFrame and check top and bottom of the data

In [4]:
may_df = pd.read_pickle('../data/may.pkl')
may_df.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
0,2019-05-01 00:01:41.247000,36.136822,-86.799877,PoweredLIRL1,93.0,Bird
1,2019-05-01 00:01:41.247000,36.191252,-86.772945,PoweredXWRWC,35.0,Bird
2,2019-05-01 00:01:41.247000,36.144752,-86.806293,PoweredMEJEH,90.0,Bird
3,2019-05-01 00:01:41.247000,36.162056,-86.774688,Powered1A7TC,88.0,Bird
4,2019-05-01 00:01:41.247000,36.150973,-86.783109,Powered2TYEF,98.0,Bird


In [5]:
may_df.tail()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
20292498,2019-05-31 23:59:42,36.1639,-86.7823,Poweredbb60ef04-c719-597f-805f-20978964fd8c,49.0,Jump
20292499,2019-05-31 23:59:42,36.1606,-86.7768,Powered9f5743ed-7f81-539b-9e37-6503d54445d3,68.0,Jump
20292500,2019-05-31 23:59:42,36.1606,-86.7769,Powered5744d1db-3bbe-55e5-9a23-a93828ba54fc,53.0,Jump
20292501,2019-05-31 23:59:42,36.1624,-86.7745,Poweredad1d6d3b-162a-5bec-bffd-cac4ac8dee1c,42.0,Jump
20292502,2019-05-31 23:59:42,36.1519,-86.7985,Powered8505008e-97c9-5003-8975-2ea6b2634fa0,76.0,Jump


#### Check column attributes

In [6]:
may_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20283582 entries, 0 to 20292502
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   chargelevel  float64
 5   companyname  object 
dtypes: float64(3), object(3)
memory usage: 1.1+ GB


#### Convert pubdatetime column from string to datetime format

In [12]:
may_df['pubdatetime'] = pd.to_datetime(may_df['pubdatetime'])
may_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20283582 entries, 0 to 20292502
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 1.1+ GB


#### Check for null within columns

In [8]:
may_df.isnull().sum()

pubdatetime      0
latitude         0
longitude        0
sumdid           0
chargelevel    283
companyname      0
dtype: int64

In [9]:
may_df.companyname.value_counts()

Lime      7111655
Bird      4334006
Lyft      3112581
Jump      2984630
Spin      1301001
Gotcha     915314
Bolt       524395
Name: companyname, dtype: int64

#### Check for dups within sumdid

In [10]:
may_df.sumdid.nunique()

8303

In [11]:
may_df.describe()

,latitude,longitude,chargelevel
count,2.028358e+07,2.028358e+07,2.028330e+07
mean,3.614047e+01,-8.675242e+01,6.590530e+01
std,6.193919e-01,1.486410e+00,3.041610e+01
min,0.000000e+00,-8.728604e+01,0.000000e+00
25%,3.614305e+01,-8.679267e+01,4.700000e+01
50%,3.615411e+01,-8.678090e+01,7.200000e+01
75%,3.616320e+01,-8.677170e+01,9.300000e+01
max,3.661670e+01,0.000000e+00,1.000000e+02
